# This script processes the o4 minradius results for best level-minRadius configuration for each cloud

1. Compile o3memo with:
```
make bin/o3memo 
```
2. Run the benchmarking script
```
cd scripts
python3 run_par_speedup.py
```
3. The output that we will process is `o4_speedup_<hostname>.csv` that should be already saved in the `Results` folder.


In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the files exist
file_list = [                   
                f'o4_speedup_{hostname}.csv',
            ]

for o4_file in file_list:
    # if it is not in Results
    if not os.path.exists(o4_file):
        # is the file already in the directory?
        assert os.path.exists(os.path.join(hostname, o4_file)), f'File {o4_file} not found: something went wrong with the baseline benchmark.'
    # if it is in Results
    else:
        # copy the file to the directory
        assert os.system(f'mv {o4_file} {hostname}/') == 0, f'Failed to move {o4_file} to {hostname}/'
# add the path to all the names in the list
for i in range(len(file_list)):
    file_list[i] = os.path.join(hostname, file_list[i])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean

def tokenize(filename):
    results ={}
    with open(filename) as f:
        next(f) # skip header
        for line in f:
            tokens = line.split()
            name=tokens[1].split("/")[3][:-5]
            nth=int(tokens[2])
            level=int(tokens[5])
            if name not in results:
                results[name]={}
            if level not in results[name]:
                results[name][level]={}
            results[name][level][nth]=[float(tokens[6])] # tree construction time
            results[name][level][nth].append(float(tokens[7])) # owm time
            results[name][level][nth].append(float(tokens[6])+float(tokens[7])) # total time
            results[name][level][nth].append(float(tokens[8])) # accuracy
    return results

resSP = tokenize(os.path.join(hostname, f'o4_speedup_{hostname}.csv'))


In [ ]:
#Configuration variables
def plot_res(res):
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8

    fig = plt.figure()

    marks=['o-','x-','s-','v-','+-']
    # there is only one possible level for each cloud, as we performed the tests with only one level
    bestlevel=[min(res[i], key=lambda x: res[i][x][8][2]) for i in res]
    numthreads=list(res['Alcoy'][bestlevel[0]].keys())
    print(bestlevel,numthreads)
    labels=['Tree Const.','OWM Trav.','Total']

    fig, axs = plt.subplots(nrows=1, ncols=3,figsize=(15, 5), constrained_layout=True)
    sizes=[i+" (lev="+str(j)+")" for i,j in zip(res,bestlevel)]
    sizes.append('Ideal')

    for phase,subfig in zip(['qtree','owm','total'],range(3)):
        # print("Phase: "+phase+ " speedup for each cloud and best level for each cloud: "+str(list(res.keys())))
        for (i,z,lev) in zip(res,marks,bestlevel):
            # print(np.array([res[i][lev][1][subfig]/res[i][lev][j][subfig] for j in numthreads]))
            axs[subfig].plot(np.array(numthreads), np.array([res[i][lev][1][subfig]/res[i][lev][j][subfig] for j in numthreads]), z, linewidth=linew, markersize=markers)
        axs[subfig].plot(np.array(numthreads), np.array(numthreads), '-', linewidth=linew, markersize=markers)
        axs[subfig].set_title(f'{labels[subfig]}',  fontweight='bold', fontsize=ylabelfs)
        axs[subfig].set_xlabel('Number of cores', fontsize=xlabelfs)
        axs[subfig].set_xticks(numthreads)
        axs[subfig].tick_params(axis='x', labelsize=xticksfs)
        axs[subfig].grid()

    fig.suptitle(f'Speedup for each cloud and each phase O4 @ {hostname}', fontweight='bold', fontsize=titlefs)
    axs[0].set_ylabel('Speedup', fontsize=ylabelfs)
    axs[0].legend(sizes,loc='best', fontsize= legendfs)

    return fig

fig = plot_res(resSP)

In [ ]:
pp = PdfPages(f"{hostname}/Speedup-o4-{hostname}.pdf")
pp.savefig(fig)
pp.close()